In [1]:
import enum
import numpy as np

class Symbol(enum.IntEnum):
    SCATTER = 1
    WILD = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10
    ELEVEN = 11

class Win(enum.IntEnum):
    THREE = 3
    FOUR = 4
    FIVE = 5

class SlotMachine:
    """
        A model of a slot machine
    """

    def __init__(self, reels = [], height = 3, payoff = np.zeros((len(Symbol) + 1, 5)), free_spins = []):
        """
        C-tor
        :param height: int (default 3)
                        the height of a window
        """

        self.reels = reels
        self.height = height
        self.rng = np.random.default_rng(seed = 2022)
        self.payoff = payoff
        self.free_spins = free_spins

    def play(self):
        window = np.zeros((self.height, len(self.reels)))
        pos = self.rng.integers(0, len(self.reels[0]),
                                size = len(self.reels),
                                dtype = np.int32,
                                endpoint = False)
        for i in range(len(self.reels)):
            for j in range(self.height):
                window[j][i] = self.reels[i][(pos[i] + j) % len(self.reels[i])]
        return window

    def get_prob(self, type_win = Win.FIVE, symbol = Symbol.THREE):
        """
        Compute probability of the specific combination on reels
        :param symbol:
        :param type_win:
        :return: float,
                    probability of the combination
        """

        iterations = 100000
        count = 0.0;
        for i in range(iterations):
            window = self.play()
            flag = True
            for j in range(type_win.value):
                if not np.any(
                        np.isin(
                            [symbol.value, Symbol.WILD.value],
                            window[:, j]
                        )
                ):
                    flag = False
                    break;
            if (type_win.value >= len(self.reels) or
                not np.any(
                    np.isin(
                        [symbol.value, Symbol.WILD.value],
                        window[:, type_win.value])
                    )
                ) and flag:
                count += 1
        return count / iterations

    def test_prob(self):
        probs = np.zeros((len(Win), len(Symbol) - 2))
        for i in range(len(Win)):
            for j in range(len(Symbol) - 2):
                probs[i, j] = self.get_prob(Win(Win.THREE + i),
                                            Symbol(Symbol.THREE + j))
        return probs

    def pay(self, window):
        m, n = window.shape
        winline = 0
        for i in range(m):
            symbol = Symbol(window[i][0])
            j = 0
            while j < n and np.any(
                    np.isin(
                        [symbol, Symbol.WILD.value],
                        window[:, j])
                    ):
                j+=1
            winline += self.payoff[symbol][j - 1]
        return winline

    def mean_pay(self):
        iterations = 100000
        sum = 0.0
        for _ in range(iterations):
            sum += self.pay(self.play())
        return sum / iterations

    def stat_scatters(self):
        """
        Compute statistics for scatters
        :return: mean number of spins without free spins, mean free spins
        """
        iterations = 100000
        spins = []
        r = 0.0
        bonus_spins = 0.0
        for i in range(iterations):
            window = self.play()
            r += 1
            j = sum(np.count_nonzero(line == Symbol.SCATTER.value) for line in window)
            if j >= 3:
                spins.append(r)
                r = 0
            for k, v in self.free_spins.items():
                bonus_spins += int(j / k) * v
                j = j % k
        spins.append(r)
        # print(spins)
        return np.mean(spins), bonus_spins / iterations

We create an instance of class `SlotMachine` based on data from task 2

In [2]:
reel_1 = [10, 1, 4, 5, 8, 11, 4, 5, 9, 8, 7, 11, 10, 6, 7, 9, 8, 10, 3, 9, 11, 6, 9]
reel_2345 = [6, 1, 3, 8, 7, 6, 9, 11, 2, 8, 4, 5, 10, 11, 4, 10, 8, 7, 9, 11, 5, 10, 9]
payoff = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 2000, 10000, 20000],
    [0, 0, 1000, 5000, 10000],
    [0, 0, 1000, 4000, 6000],
    [0, 0, 1000, 2000, 5000],
    [0, 0, 500, 1000, 4000],
    [0, 0, 500, 1000, 4000],
    [0, 0, 100, 500, 2000],
    [0, 0, 100, 500, 2000],
    [0, 0, 100, 500, 2000]
]
free_spins = {5: 10, 4: 8, 3: 6}
sm = SlotMachine([reel_1, reel_2345, reel_2345, reel_2345, reel_2345], payoff = payoff, free_spins=free_spins)

We find probabilities of winning combinations. In the obtained matrix the first column contains probabilities of the winning combinations with the first three symbols of 2 or 3, with the first four symbols of 2 or 3 and all five symbols of 2 or 3. The second column consists of probabilities of the winning combinations with the first three symbols of 2 or 4, with the first four symbols of 2 or 4 and the five symbols of 2 or 4 etc.

In [3]:
print(sm.test_prob())

[[0.00658 0.02028 0.02445 0.02457 0.02426 0.04151 0.06164 0.05175 0.04125]
 [0.00177 0.00711 0.00945 0.0093  0.00938 0.01844 0.02992 0.02704 0.01711]
 [0.00054 0.00428 0.00617 0.00613 0.00611 0.01433 0.02705 0.02888 0.01471]]


We find mean payback for a player

In [4]:
sm.mean_pay()

648.23

We find the mean number of spins without bonus spins and mean number of free spins

In [5]:
sm.stat_scatters()

(57.736720554272516, 0.10644)